In [2]:
from sklearn.model_selection import train_test_split #for data splitting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier

In [3]:
def preprocess(countpath, testsize = 0.2):
    x = pd.read_csv('%s' % countpath, index_col=0).T

    # get feature names
    featurenames = np.array(x.index)

    # get label names
    x = x.T
    y = list(np.array(x.index))
    label = sorted(list(set(y)), key = y.index)
    
    df = x
    df['target'] = y

    for classname in label:
        count_temp = len(df[df.target == classname])
        print("Proportion of family {0}: {1:.2f}%".format(classname, (count_temp / (len(df.target)) * 100)))


    y = df.target.values
    x= df.drop(['target'], axis=1)
    
    # Normalize
#     x= x.apply(lambda x :(x-np.min(x))/(np.max(x)-np.min(x)))
    
    x = x.dropna(axis=1, how='any')  # drop all rows that have any NaN value
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = testsize, random_state = 5)

    return x_train, y_train, x_test, y_test, featurenames, label, x

In [ ]:
x_train, y_train, x_test, y_test, feature_names, _, x = preprocess('156_new_selected_156.csv', 0.3)

In [46]:
dict = {}
for i in label:
    if i == 'RF00168' or i == 'RF00167':
        dict[i] = 7
    elif i == 'RF01054' or i == 'RF00522':
        dict[i] = 150
    elif i == 'RF00634' or i == 'RF00521' or i == 'RF00234':
        dict[i] = 30
    elif i == 'RF01055' or i == 'RF01057' or i == 'RF01063':
        dict[i] = 15
    elif i == 'RF00050' or i == 'RF00162' or i == 'RF01051' or i == 'RF00162':
        dict[i] = 3
    else:
        dict[i] = 1

model = RandomForestClassifier(n_jobs=48, n_estimators = 10000, max_depth = 20, class_weight = dict)
model.fit(x_train, y_train)

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]


final_labels = []
for f in range(x_train.shape[1]):
    final_labels.append(feature_names[indices[f]])
    print("%2d) %-*s %f" % (f + 1, 30, final_labels[f], importances[indices[f]]))
print(final_labels)


for i in {20, 50, 100}:
    final = final_labels[0:i]
    x_temp = x
    for j in feature_names:
        flag = 0
        for k in final:
            if j == k:
                flag = 1
        if flag == 0:
            x_temp = x_temp.drop([j.upper()], axis = 1)
#     print(x_temp)
    x_temp.to_csv('156_new_selected_%s.csv' % i, index=True)